In [4]:
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
import pandas as pd
import fitz  # PyMuPDF for reading PDF files
import os  # For file path checking
from DataAnalyzer import DataAnalyzer  # استيراد DataAnalyzer

# Define FAISS database path
FAISS_DB_PATH = "faiss_index"

# 1️⃣ Load rules from PDF memory
def load_analysis_rules_from_memory(pdf_content):
    doc = fitz.open(stream=pdf_content, filetype="pdf")
    documents = [Document(page_content=page.get_text()) for page in doc]
    return documents

# 2️⃣ Train RAG system with FAISS
def train_rag_system(documents):
    """Train or load the RAG model with FAISS to avoid recomputation."""
    embedding_model = OllamaEmbeddings(model="llama2")
    
    if os.path.exists(FAISS_DB_PATH):
        print("\n🔄 Loading existing FAISS index...")
        vector_db = FAISS.load_local(
            FAISS_DB_PATH, 
            embedding_model, 
            allow_dangerous_deserialization=True
        )
    else:
        print("\n🛠️ Generating new embeddings and saving FAISS index...")
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=300)
        texts = text_splitter.split_documents(documents)
        vector_db = FAISS.from_documents(texts, embedding_model)
        vector_db.save_local(FAISS_DB_PATH)
    
    retriever = vector_db.as_retriever(search_type="mmr", search_kwargs={"k": 5})
    llm = Ollama(model="llama2")
    
    retrievalQA = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True
    )
    
    return retrievalQA, llm

# 3️⃣ Load CSV file
def load_csv(file_path):
    return pd.read_csv(file_path)

# 🚀 Main execution
if __name__ == "__main__":
    # Load rules from PDF file
    pdf_file_path = "storying.pdf"
    if not os.path.exists(pdf_file_path):
        raise FileNotFoundError(f"🚨 Error: The file '{pdf_file_path}' was not found!")
    
    with open(pdf_file_path, "rb") as file:
        pdf_content = file.read()
    
    documents = load_analysis_rules_from_memory(pdf_content)
    
    # Train RAG model
    retrievalQA, llm = train_rag_system(documents)
    
    # Load CSV data
    csv_file_path = "Regions.csv"
    if not os.path.exists(csv_file_path):
        raise FileNotFoundError(f"🚨 Error: The file '{csv_file_path}' was not found!")

    df = load_csv(csv_file_path)
    
    # Create DataAnalyzer instance
    analyzer = DataAnalyzer(df, llm=llm)
    
    # Perform data analysis and generate query
    analysis_result, questions = analyzer.analysis_data()
    print("\n📊 Analysis Result:")
    print(analysis_result)
    
    # Send each question to RAG Agent
    for question in questions:
        print(f"\n❓ Question: {question}")
        result = retrievalQA.invoke({"query": question})
        print("🔍 Answer:")
        print(result['result'])
        print("\n📚 Source Documents:")
        for doc in result['source_documents']:
            print(doc.page_content)


🔄 Loading existing FAISS index...


ValueError: too many values to unpack (expected 2)